# Assignment 1: Wrangling and EDA
### Foundations of Machine Learning

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `airbnb_NYC.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing? For the remaining missing values, replace the `np.nan/None` values with the label `Missing`.
3. Dummy variable: For `metabric.csv`, convert the `Overall Survival Status` variable into a dummy/binary variable, taking the value 0 if the patient is deceased and 1 if they are living.
4. Missing values: For `airbnb_NYC.csv`, determine how many missing values of `Review Scores Rating` there are. Create a new variable, in which you impute the median score for non-missing observations to the missing ones. Why might this bias or otherwise negatively impact your results?

In [6]:
import urllib.request
import os
import zipfile
import os

def download_data(force=False):
    """Download and extract course data from Zenodo."""

    zip_path = 'data.zip'
    data_dir = './data'

    if not os.path.exists(zip_path) or force:
        print("Downloading course data...")
        urllib.request.urlretrieve(
            'https://zenodo.org/records/18235955/files/data.zip?download=1',
            zip_path
        )
        print("Download complete")

    if not os.path.exists(data_dir) or force:
        print("Extracting data files...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        print("Data extracted")

    return data_dir


if __name__ == "__main__":
    download_data()


Download complete
Extracting data files...
Data extracted


In [8]:
os.listdir()


['.config', 'data.zip', 'data', 'sample_data']

In [9]:
os.listdir("data")

['electricity_data_validation.csv',
 'CardiacPatientData.csv',
 'VirginiaPretrialData2017Codebook.pdf',
 'tuna.csv',
 'crimebystatecombinedwithunemployment.csv',
 'ForeignGifts_edu.csv',
 'iowa.csv',
 'corporate_ratings.csv',
 'time_use.csv',
 'sharks.csv',
 'metabric.csv',
 'USA_cars_datasets.csv',
 'airbnb_hw.csv',
 'energy.csv',
 'mn_police_use_of_force.csv',
 'craiglist_cville_cars_long.csv',
 'tech_stocks.csv',
 'cville_weather.csv',
 'ames_codebook.txt',
 'nhanes_data_17_18.csv',
 'wages_hw.csv',
 'craigslist_cville_cars.csv',
 'cars_hw.csv',
 'weapons-trade-register.csv',
 'mammogram.csv',
 'us_power_consumption.csv',
 'divorce',
 'airbnb_NYC.csv',
 'nhanes_meta_17_18.csv',
 'pierce_county_house_sales.csv',
 'heart_failure_clinical_records_dataset.csv',
 'heart_failure_readme.txt',
 'mammogram.txt',
 'land_mines.csv',
 'justice_data.parquet',
 'mnist',
 'drilling_rigs.csv',
 'use_of_force.txt',
 'brazil_default.csv',
 'Effectiveness of Bystander Cardiopulmonary Resuscitation and

In [66]:
# Load Airbnb data
airbnb = pd.read_csv("data/airbnb_NYC.csv", encoding="latin-1")

# Clean price variable
airbnb["price_clean"] = (
    airbnb["Price"]
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
    .astype(float)
)

In [65]:
# Count missing values
airbnb["price_clean"].isna().sum()


np.int64(0)

1.1 The price variable was stored as a string because it included dollar signs and commas. I removed the dollar signs and commas before converting the variable to a numeric type. Commas cause prices above $999 to be incorrectly parsed as missing values during conversion. After cleaning and converting the variable to a float, there were 0 missing values in the cleaned price variable.

In [63]:
# Load MN police use of force data
mn = pd.read_csv("data/mn_police_use_of_force.csv")

# Clean subject_injury
mn["subject_injury_clean"] = (
    mn["subject_injury"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map({"yes": "Yes", "no": "No"})
)

In [64]:
# Proportion missing
prop_missing = mn["subject_injury_clean"].isna().mean()
prop_missing


np.float64(0.7619342359767892)

In [34]:
# Cross-tabulate with force type
injury_crosstab = pd.crosstab(
    mn["subject_injury_clean"],
    mn["force_type"],
    dropna=False
)

injury_crosstab


force_type,Baton,Bodily Force,Chemical Irritant,Firearm,Gun Point Display,Improvised Weapon,Less Lethal,Less Lethal Projectile,Maximal Restraint Technique,Police K9 Bite,Taser
subject_injury_clean,,,,,,,,,,,
Missing,2,7051,1421,0,27,74,87,0,170,31,985
No,0,1093,131,2,33,34,0,1,0,2,150
Yes,2,1286,41,0,44,40,0,2,0,44,172


In [36]:
mn["subject_injury_clean"] = mn["subject_injury_clean"].fillna("Missing")

1.2 I cleaned the subject_injury variable by standardizing values and mapping them to “Yes” and “No.” Approximately 76% of the observations are missing injury information. A cross-tabulation with force type shows that missing values are concentrated in certain force types, such as bodily force and chemical irritants, suggesting that injury reporting is not random. I replaced the remaining missing values with the label “Missing.”

In [40]:
import pandas as pd

# Load METABRIC data
metabric = pd.read_csv("data/metabric.csv")

# Create dummy variable for survival status
# 0 = Deceased, 1 = Living
metabric["survival_dummy"] = metabric["Overall Survival Status"].map({
    "1:DECEASED": 0,
    "0:LIVING": 1
})

# Check result
metabric["survival_dummy"].value_counts()


,count
survival_dummy,
0,768
1,575


In [42]:
missing_reviews = airbnb["Review Scores Rating"].isna().sum()
missing_reviews

np.int64(8323)

In [44]:
# Compute median of non-missing values
median_review_score = airbnb["Review Scores Rating"].median()

# Create new variable with median imputation
airbnb["Review_Scores_Rating_imputed"] = airbnb["Review Scores Rating"].fillna(
    median_review_score
)

# Check that missing values are now filled
airbnb["Review_Scores_Rating_imputed"].isna().sum()

np.int64(0)

1.4 There are 8323 missing values in the Review Scores Rating Variable. Median imputation may bias results because it reduces variability in the data and assumes that listings with missing review scores are similar to typical listings. If missing values are related to listing quality or lack of reviews, this approach could distort relationships involving review scores.

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work. What does work?
2. Drop any columns that do not contain data.
3. What is an observation? Carefully justify your answer, and explain how it affects your choices in cleaning and analyzing the data.
4. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
5. Clean the Age variable and make a histogram of the ages of the victims.
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Thoughts?

In [49]:
import pandas as pd

# Load shark attack data (Excel file)
shark = pd.read_excel("data/GSAF5.xls")

# Preview data
shark.head()


,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,20th January,2026.0,Unprovoked,Australia,NSW,Point Plomber North of Port Macquarie,Surfing,Paul Zvirdinas,M,39,...,Bull shark,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19th January,2026.0,Unprovoked,Australia,NSW,Dee Why,Surfing,Unknown,M,11,...,Bull shark,Andy Currie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19th January,2026.0,Unprovoked,Australia,NSW,North Steyne,Surfing,Andre de Ruyter,M,27,...,5m shark species not determined,9 News: Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18th January,2026.0,Unprovoked,Australia,NSW,Nielsen Park Vaucluse Sydney Harbour,Jumping off rocks,Nico Antic,M,12,...,Bull shark,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10th January,2026.0,Unprovoked,Australia,NSW,Avalon Beach,Surfing,Paul Stanton,M,?,...,Unknown,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2.1 The shark attack dataset is provided as an Excel (.xls) file rather than a CSV, so pd.read_csv() does not work. Instead, I used pd.read_excel() to successfully load the data into Pandas.

In [53]:
# Drop columns that contain no data at all
shark_clean = shark.dropna(axis=1, how="all")


In [54]:
shark.columns[shark.isna().all()]


Index([], dtype='object')

In [55]:
shark.shape, shark_clean.shape


((7070, 23), (7070, 23))

In [56]:
shark_clean.head()


,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,20th January,2026.0,Unprovoked,Australia,NSW,Point Plomber North of Port Macquarie,Surfing,Paul Zvirdinas,M,39,...,Bull shark,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19th January,2026.0,Unprovoked,Australia,NSW,Dee Why,Surfing,Unknown,M,11,...,Bull shark,Andy Currie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19th January,2026.0,Unprovoked,Australia,NSW,North Steyne,Surfing,Andre de Ruyter,M,27,...,5m shark species not determined,9 News: Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18th January,2026.0,Unprovoked,Australia,NSW,Nielsen Park Vaucluse Sydney Harbour,Jumping off rocks,Nico Antic,M,12,...,Bull shark,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10th January,2026.0,Unprovoked,Australia,NSW,Avalon Beach,Surfing,Paul Stanton,M,?,...,Unknown,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2.3 An observation in this dataset is a single shark attack incident, with each row representing one recorded event. This is justified because the rows are organized by case number and date, and all variables describe details of the same incident. Defining an observation this way affects cleaning and analysis decisions by prioritizing the preservation of rows, since dropping rows would remove entire incidents. As a result, cleaning focused on removing columns with no data rather than deleting observations, ensuring analyses accurately reflect the number and characteristics of shark attack events.

In [60]:
# Remove invalid years (e.g., 0 or negative)
shark_clean = shark_clean[shark_clean["Year_clean"] > 0]

# Recheck range
shark_clean["Year_clean"].min(), shark_clean["Year_clean"].max()


(5.0, 2026.0)

In [58]:
shark_1940 = shark_clean[shark_clean["Year_clean"] >= 1940]

In [59]:
attacks_per_year = (
    shark_1940.groupby("Year_clean")
    .size()
)

attacks_per_year.tail()


,0
Year_clean,
2022.0,98
2023.0,109
2024.0,52
2025.0,66
2026.0,8


**Q3.** Open the "tidy_data.pdf" document available in `https://github.com/ds4e/wrangling`, which is a paper called *Tidy Data* by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?

**Q4.** This question looks at financial transfers from international actors to American universities. In particular, from which countries and giftors are the gifts coming from, and to which institutions are they going?

For this question, `.groupby([vars]).count()` and `.groupby([vars]).sum()` will be especially useful to tally the number of occurrences and sum the values of those occurrences.

1. Load the `ForeignGifts_edu.csv` dataset.
2. For `Foreign Gift Amount`, create a histogram and describe the variable. Describe your findings.
3. For `Gift Type`, create a histogram or value counts table. What proportion of the gifts are contracts, real estate, and monetary gifts?
4. What are the top 15 countries in terms of the number of gifts? What are the top 15 countries in terms of the amount given?
5. What are the top 15 institutions in terms of the total amount of money they receive? Make a histogram of the total amount received by all institutions.
6. Which giftors provide the most money, in total?

**Q5.** This question provides some practice doing exploratory data analysis and visualization.

We'll use the `college_completion.csv` dataset from the US Department of Education. The "relevant" variables for this question are:
  - `level` - Level of institution (4-year, 2-year)
  - `aid_value` - The average amount of student aid going to undergraduate recipients
  - `control` - Public, Private not-for-profit, Private for-profit
  - `grad_100_value` - percentage of first-time, full-time, degree-seeking undergraduates who complete a degree or certificate program within 100 percent of expected time (bachelor's-seeking group at 4-year institutions)

1. Load the `college_completion.csv` data with Pandas.
2. How many observations and variables are in the data? Use `.head()` to examine the first few rows of data.
3. Cross tabulate `control` and `level`. Describe the patterns you see in words.
4. For `grad_100_value`, create a kernel density plot and describe table. Now condition on `control`, and produce a kernel density plot and describe tables for each type of institutional control. Which type of institution appear to have the most favorable graduation rates?
5. Make a scatterplot of `grad_100_value` by `aid_value`, and compute the covariance and correlation between the two variables. Describe what you see. Now make the same plot and statistics, but conditioning on `control`. Describe what you see. For which kinds of institutions does aid seem to vary positively with graduation rates?

**Q6.** In class, we talked about how to compute the sample mean of a variable $X$,
$$
m(X) = \dfrac{1}{N} \sum_{i=1}^N x_i
$$
and sample covariance of two variables $X$ and $Y$,
$$
\text{cov}(X,Y) = \dfrac{1}{N} \sum_{i=1}^N (x_i - m(X))(y_i - m(Y))).
$$
Recall, the sample variance of $X$ is
$$
s^2 = \dfrac{1}{N} \sum_{i=1}^N (x_i - m(X))^2.
$$
It can be very helpful to understand some basic properties of these statistics. If you want to write your calculations on a piece of paper, take a photo, and upload that to your GitHub repo, that's probably easiest.

We're going to look at **linear transformations** of $X$, $Y = a + bX$. So we take each value of $X$, $x_i$, and transform it as $y_i = a + b x_i$.

1. Show that $m(a + bX) = a+b \times m(X)$.
2. Show that $ \text{cov}(X,X) = s^2$.
3. Show that $\text{cov}(X,a+bY) = b \times \text{cov}(X,Y)$
4. Show that $\text{cov}(a+bX,a+bY) = b^2 \text{cov}(X,Y) $. Notice, this also means that $\text{cov}(bX, bX) = b^2 s^2$.
5. Suppose $b>0$ and let the median of $X$ be $\text{med}(X)$. Is it true that the median of $a+bX$ is equal to $a + b \times \text{med}(X)$? Is the IQR of $a + bX$ equal to $a + b \times \text{IQR}(X)$?
6. Show by example that the means of $X^2$ and $\sqrt{X}$ are generally not $(m(X))^2$ and $\sqrt{m(X)}$. So, the results we derived above really depend on the linearity of the transformation $Y = a + bX$, and transformations like $Y = X^2$ or $Y = \sqrt{X}$ will not behave in a similar way.

**Q7.** This question provides some practice doing exploratory data analysis and visualization.

We'll use the `ames_prices.csv` dataset. The "relevant" variables for this question are:
  - `price` - Sale price value of the house
  - `Bldg.Type` - Building type of the house (single family home, end-of-unit townhome, duplex, interior townhome, two-family conversion)

1. Load the `college_completion.csv` data with Pandas.
2. Make a kernel density plot of price and compute a describe table. Now, make a kernel density plot of price conditional on building type, and use `.groupby()` to make a describe type for each type of building. Which building types are the most expensive, on average? Which have the highest variance in transaction prices?
3. Make an ECDF plot of price, and compute the sample minimum, .25 quantile, median, .75 quantile, and sample maximum (i.e. a 5-number summary).
4. Make a boxplot of price. Are there outliers? Make a boxplot of price conditional on building type. What patterns do you see?
5. Make a dummy variable indicating that an observation is an outlier.
6. Winsorize the price variable, and compute a new kernel density plot and describe table. How do the results change?